In [131]:
import pandas as pd
import numpy as np

In [145]:
data_low_test = pd.read_csv('data/interim/data_low_test.csv')
print(data_low_test['buyer_nbr'].nunique())
print(data_low_test['lot_nbr'].nunique())

23738
47332


In [146]:
data_low_test

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
0,Consumer,WI,68964035,923225,hattmuff@icloud.com,15000.0,2025-09-29,2024,TOYT,LAND CRUIS,61408.00,34094.29,10,3
1,Consumer,IN,68520435,22183,jwilliams3600@gmail.com,1400.0,2025-09-16,2006,CHRY,300,8133.00,8133.00,2,3
2,Consumer,NJ,66350705,421465,walferp@gmail.com,200.0,2025-09-09,2017,NISS,PATHFINDER,3445.00,0.00,20,3
3,Dealer,GA,62710295,535089,carismamotorsports1@gmail.com,650.0,2025-08-29,2006,BMW,5 SERIES,5876.00,4967.64,2,3
4,Dealer,MA,71169255,382343,jcorrenti@fleetfirst.com,850.0,2025-10-09,2018,FORD,TRANSIT CO,13346.93,30265.91,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57802,Dealer,NY,57054795,879827,Phonors@hotmail.com,2050.0,2025-08-29,2011,JEP,WRANGLER,12300.00,0.00,17,2
57803,Export,FL,63570705,371619,191657-MSTRBDR-85780561-aab4-499d-89e1-f7f7517...,9400.0,2025-07-25,2022,JEP,WRANGLER,32696.00,28795.34,7,6
57804,Dealer,GA,55872965,675625,leaderautomotive01@gmail.com,725.0,2025-07-16,2004,JEP,WRANGLER,11270.00,11270.00,5,5
57805,Consumer,KY,80458955,932905,roycornelius1953@gmail.com,925.0,2025-09-24,2000,JEP,WRANGLER,8026.57,5696.73,4,4


In [147]:
data_low_test.isnull().sum()

mbr_lic_type                       0
mbr_state                          0
lot_nbr                            0
buyer_nbr                          0
mbr_email                          0
max_bid                            0
inv_dt                             0
lot_year                           0
lot_make_cd                        0
grp_model                          0
acv                                0
repair_cost                        0
total_unique_buyers_on_that_lot    0
total_unique_lots_bid_by_buyers    0
dtype: int64

In [148]:
data_low_test = data_low_test[['buyer_nbr','mbr_email','lot_nbr', 'lot_year', 'lot_make_cd', 'grp_model', 'acv', 'repair_cost', 'inv_dt']]
data_low_test

,buyer_nbr,mbr_email,lot_nbr,lot_year,lot_make_cd,grp_model,acv,repair_cost,inv_dt
0,923225,hattmuff@icloud.com,68964035,2024,TOYT,LAND CRUIS,61408.00,34094.29,2025-09-29
1,22183,jwilliams3600@gmail.com,68520435,2006,CHRY,300,8133.00,8133.00,2025-09-16
2,421465,walferp@gmail.com,66350705,2017,NISS,PATHFINDER,3445.00,0.00,2025-09-09
3,535089,carismamotorsports1@gmail.com,62710295,2006,BMW,5 SERIES,5876.00,4967.64,2025-08-29
4,382343,jcorrenti@fleetfirst.com,71169255,2018,FORD,TRANSIT CO,13346.93,30265.91,2025-10-09
...,...,...,...,...,...,...,...,...,...
57802,879827,Phonors@hotmail.com,57054795,2011,JEP,WRANGLER,12300.00,0.00,2025-08-29
57803,371619,191657-MSTRBDR-85780561-aab4-499d-89e1-f7f7517...,63570705,2022,JEP,WRANGLER,32696.00,28795.34,2025-07-25
57804,675625,leaderautomotive01@gmail.com,55872965,2004,JEP,WRANGLER,11270.00,11270.00,2025-07-16
57805,932905,roycornelius1953@gmail.com,80458955,2000,JEP,WRANGLER,8026.57,5696.73,2025-09-24


In [149]:
data_low_test.rename(columns={
    'buyer_nbr': 'input_buyer_nbr',
    'lot_nbr': 'recommended_lot'
}, inplace=True)

/var/folders/jj/1fql_rw17h10wm_w98gwx9wm0000gq/T/ipykernel_41740/1835412636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low_test.rename(columns={


In [150]:
data2 = pd.read_csv('data/interim/future_lots.csv')

In [151]:
data2.isnull().sum()

lot_nbr             0
lot_year            0
lot_make_cd         0
grp_model           0
damage_type_desc    0
repair_cost         0
acv                 0
auc_dt              0
proquote_amt        0
dtype: int64

In [152]:
def recommend_lots_for_buyer(buyer_id, buyer_lots_df, upcoming_df, top_k=6):
    from sklearn.metrics.pairwise import manhattan_distances
    import numpy as np
    import pandas as pd

    results = []
    used_lots = set()

    # ------------------------
    # STEP 1: One-to-one exact YMM + fallback
    # ------------------------
    for _, row in buyer_lots_df.iterrows():
        acv, repair = row['acv'], row['repair_cost']
        year, make, model = row['lot_year'], row['lot_make_cd'], row['grp_model']
        input_vec = np.array([[acv, repair]])

        ymm_candidates = upcoming_df[
            (upcoming_df['lot_year'] == year) &
            (upcoming_df['lot_make_cd'] == make) &
            (upcoming_df['grp_model'] == model)
        ][['lot_nbr', 'acv', 'repair_cost']].dropna()

        if ymm_candidates.empty:
            ymm_candidates = upcoming_df[['lot_nbr', 'acv', 'repair_cost']].dropna()

        ymm_candidates = ymm_candidates[~ymm_candidates['lot_nbr'].isin(used_lots)]
        if ymm_candidates.empty:
            continue

        ymm_candidates['manhattan_dist'] = manhattan_distances(
            ymm_candidates[['acv', 'repair_cost']].values, input_vec
        ).flatten()

        best_match = ymm_candidates.sort_values('manhattan_dist').iloc[0]

        results.append({
            'input_buyer_nbr': buyer_id,
            'original_lot': int(row['recommended_lot']),
            'recommended_lot': int(best_match['lot_nbr']),
            'manhattan_distance': float(best_match['manhattan_dist']),
            'source': 'Step 1 - YMM/Manhattan'
        })
        used_lots.add(int(best_match['lot_nbr']))

    # ------------------------
    # STEP 2: Try YMM on most recent interacted lot
    # ------------------------
    if len(results) < top_k:
        most_recent = buyer_lots_df.sort_values('inv_dt', ascending=False).iloc[0]
        acv, repair = most_recent['acv'], most_recent['repair_cost']
        year, make, model = most_recent['lot_year'], most_recent['lot_make_cd'], most_recent['grp_model']
        input_vec = np.array([[acv, repair]])

        ymm_candidates = upcoming_df[
            (upcoming_df['lot_year'] == year) &
            (upcoming_df['lot_make_cd'] == make) &
            (upcoming_df['grp_model'] == model)
        ][['lot_nbr', 'acv', 'repair_cost']].dropna()

        ymm_candidates = ymm_candidates[~ymm_candidates['lot_nbr'].isin(used_lots)]

        if not ymm_candidates.empty:
            ymm_candidates['manhattan_dist'] = manhattan_distances(
                ymm_candidates[['acv', 'repair_cost']].values, input_vec
            ).flatten()

            for _, r in ymm_candidates.sort_values('manhattan_dist').iterrows():
                results.append({
                    'input_buyer_nbr': buyer_id,
                    'original_lot': int(most_recent['recommended_lot']),
                    'recommended_lot': int(r['lot_nbr']),
                    'manhattan_distance': float(r['manhattan_dist']),
                    'source': 'Step 2 - Recent YMM/Manhattan'
                })
                used_lots.add(int(r['lot_nbr']))
                if len(results) >= top_k:
                    break

    # ------------------------
    # STEP 3: Global MAKE match on most recent lot
    # ------------------------
    if len(results) < top_k:
        acv, repair = most_recent['acv'], most_recent['repair_cost']
        make = most_recent['lot_make_cd']
        input_vec = np.array([[acv, repair]])

        make_candidates = upcoming_df[
            (upcoming_df['lot_make_cd'] == make)
        ][['lot_nbr', 'acv', 'repair_cost']].dropna()

        make_candidates = make_candidates[~make_candidates['lot_nbr'].isin(used_lots)]

        if not make_candidates.empty:
            make_candidates['manhattan_dist'] = manhattan_distances(
                make_candidates[['acv', 'repair_cost']].values, input_vec
            ).flatten()

            for _, r in make_candidates.sort_values('manhattan_dist').iterrows():
                results.append({
                    'input_buyer_nbr': buyer_id,
                    'original_lot': int(most_recent['recommended_lot']),
                    'recommended_lot': int(r['lot_nbr']),
                    'manhattan_distance': float(r['manhattan_dist']),
                    'source': 'Step 3 - Global Make/Manhattan'
                })
                used_lots.add(int(r['lot_nbr']))
                if len(results) >= top_k:
                    break

    # ------------------------
    # STEP 4: Global fallback - No filters at all
    # ------------------------
    if len(results) < top_k:
        acv, repair = most_recent['acv'], most_recent['repair_cost']
        input_vec = np.array([[acv, repair]])

        global_candidates = upcoming_df[['lot_nbr', 'acv', 'repair_cost']].dropna()
        global_candidates = global_candidates[~global_candidates['lot_nbr'].isin(used_lots)]

        if not global_candidates.empty:
            global_candidates['manhattan_dist'] = manhattan_distances(
                global_candidates[['acv', 'repair_cost']].values, input_vec
            ).flatten()

            for _, r in global_candidates.sort_values('manhattan_dist').iterrows():
                results.append({
                    'input_buyer_nbr': buyer_id,
                    'original_lot': int(most_recent['recommended_lot']),
                    'recommended_lot': int(r['lot_nbr']),
                    'manhattan_distance': float(r['manhattan_dist']),
                    'source': 'Step 4 - Global Fallback Manhattan'
                })
                used_lots.add(int(r['lot_nbr']))
                if len(results) >= top_k:
                    break

    return results


In [153]:
def refine_recommendations_parallel_per_buyer(reco_df, data2_df, max_workers=4):
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from tqdm import tqdm

    results = []
    futures = []

    grouped = list(reco_df.groupby('input_buyer_nbr'))

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for buyer_id, group_df in grouped:
            futures.append(executor.submit(recommend_lots_for_buyer, buyer_id, group_df, data2_df))

        for f in tqdm(as_completed(futures), total=len(futures), desc="Refining recos"):
            try:
                results.extend(f.result())  # each result is a list of 6 recos
            except Exception as e:
                print(f"⚠️ Skipped buyer due to error: {e}")

    return pd.DataFrame(results)

In [154]:
# final input:
# - data_low_test → contains columns: input_buyer_nbr, recommended_lot, acv, repair_cost, lot_year, lot_make_cd, grp_model, inv_dt
# - future_lots → upcoming inventory to match against

recommended_upcoming_df_lt6 = refine_recommendations_parallel_per_buyer(data_low_test, data2, max_workers=8)


Refining recos: 100%|██████████| 23738/23738 [03:09<00:00, 125.15it/s]


In [155]:
recommended_upcoming_df_lt6.isnull().sum()

input_buyer_nbr       0
original_lot          0
recommended_lot       0
manhattan_distance    0
source                0
dtype: int64

In [156]:
recommended_upcoming_df_lt6['input_buyer_nbr'].nunique()

23738

In [157]:
recommended_upcoming_df_lt6.to_excel('data/processed/recommended_onetoone_test.xlsx', index=False)

### Reco for holdout

In [158]:
data_low_holdout = pd.read_csv('data/interim/data_low_holdout.csv')
data_low_holdout.head()

,mbr_lic_type,mbr_state,lot_nbr,buyer_nbr,mbr_email,max_bid,inv_dt,lot_year,lot_make_cd,grp_model,acv,repair_cost,total_unique_buyers_on_that_lot,total_unique_lots_bid_by_buyers
0,Consumer,CO,60941625,579924,bellarosemurphy3@gmail.com,1100.0,2025-08-05,2024,SUBA,LEGACY,25738.0,20312.37,4,2
1,Consumer,IA,64494735,505682,505682cprt_dmmy_MASTERBDR_395681_156000@copart...,5500.0,2025-09-10,2020,VOLK,ATLAS,17192.0,11897.91,5,6
2,Consumer,FL,80332155,866248,artisanaglobal@gmail.com,2300.0,2025-10-07,2013,MERZ,M-CLASS,6646.0,0.00,6,2
3,Consumer,CA,55231425,931062,931062cprt_dmmy_MASTERBDR_608649_64771@copart.com,250.0,2025-09-19,2017,MAZD,3,17501.0,0.00,5,6
4,Consumer,TX,67533135,799396,cargutcas26@hotmail.com,1950.0,2025-09-24,2015,KIA,SPORTAGE,5303.0,8699.11,2,2


In [159]:
data_low_holdout['buyer_nbr'].nunique()

23883

In [160]:
data_low_holdout.isnull().sum()

mbr_lic_type                       0
mbr_state                          0
lot_nbr                            0
buyer_nbr                          0
mbr_email                          0
max_bid                            0
inv_dt                             0
lot_year                           0
lot_make_cd                        0
grp_model                          0
acv                                0
repair_cost                        0
total_unique_buyers_on_that_lot    0
total_unique_lots_bid_by_buyers    0
dtype: int64

In [161]:
data_low_holdout = data_low_holdout[['buyer_nbr','mbr_email','lot_nbr', 'lot_year', 'lot_make_cd', 'grp_model', 'acv', 'repair_cost', 'inv_dt']]
data_low_holdout

,buyer_nbr,mbr_email,lot_nbr,lot_year,lot_make_cd,grp_model,acv,repair_cost,inv_dt
0,579924,bellarosemurphy3@gmail.com,60941625,2024,SUBA,LEGACY,25738.0,20312.37,2025-08-05
1,505682,505682cprt_dmmy_MASTERBDR_395681_156000@copart...,64494735,2020,VOLK,ATLAS,17192.0,11897.91,2025-09-10
2,866248,artisanaglobal@gmail.com,80332155,2013,MERZ,M-CLASS,6646.0,0.00,2025-10-07
3,931062,931062cprt_dmmy_MASTERBDR_608649_64771@copart.com,55231425,2017,MAZD,3,17501.0,0.00,2025-09-19
4,799396,cargutcas26@hotmail.com,67533135,2015,KIA,SPORTAGE,5303.0,8699.11,2025-09-24
...,...,...,...,...,...,...,...,...,...
58148,732778,orlandoramirezinvesting@gmail.com,67617635,2012,JEP,WRANGLER,0.0,0.00,2025-08-25
58149,747652,country_hunter1880@yahoo.com,68879115,2008,JEP,WRANGLER,11142.0,0.00,2025-09-19
58150,204718,offshoretrailers@gmail.com,73473484,2015,JEP,WRANGLER,23500.0,0.00,2025-08-19
58151,469586,vahid1dean@gmail.com,71774825,2021,JEP,WRANGLER,0.0,0.00,2025-10-02


In [167]:
data_low_holdout.rename(columns={
    'buyer_nbr': 'input_buyer_nbr',
    'lot_nbr': 'recommended_lot'
}, inplace=True)

In [168]:
recommended_upcoming_df_lt6_holdout = refine_recommendations_parallel_per_buyer(data_low_holdout, data2, max_workers=8)


Refining recos: 100%|██████████| 23883/23883 [03:16<00:00, 121.33it/s]


In [170]:
recommended_upcoming_df_lt6_holdout.to_excel('data/would_have/recommended_onetoone_holdout.xlsx', index=False)

In [169]:
recommended_upcoming_df_lt6_holdout['input_buyer_nbr'].nunique()

23883